In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import os

os.chdir(r'J:\Estudo\Projetos\VoiceToInsight')

# Changing the default cache directory of the models
os.environ['HF_HOME'] = r'J:\HuggingFace\cache' # For newer versions of transformers (>= v5)
os.environ['TRANSFORMERS_CACHE'] = r'J:\HuggingFace\cache' # For older versions of transformers

j:\Estudo\Projetos\VoiceToInsight\.venv\Lib\site-packages\transformers\utils\hub.py:125: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA GeForce RTX 3060 Ti is available.


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
recog = pipe(r'audio_samples\2963847.wav', generate_kwargs={"language": "portuguese"})

j:\Estudo\Projetos\VoiceToInsight\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [9]:
recog

{'text': ' Olá, você ligou para a Brico Bread. Estamos há mais de 25 anos levando o melhor da padaria para dentro de sua casa. Pão crocante e quentinho a qualquer hora. Digite 1 se você quer falar com vendas. 2 pós-vendas. 3 cadastro. 4 qualidade. Cadastro. 4. Qualidade. 5. Compras. 6. Recursos humanos. 7. Logística. 8. Tecnologia. Você conhece o nosso portfólio de produtos? Acesse www.bricobred.com RH Brico, boa tarde Raquel. Boa tarde Raquel, meu nome é Viviane. Eu falo aqui da TOTOS Matriz, tudo bem? Tudo bem. Eu gostaria por favor de falar com o Walter Pereira. Tá, só um momento. Tudo bem. É da? Da TOTOS. Tá, um momento. Tudo bem. É da? Da TOTOS. Tá, um momento. Viviane. Oi, pode falar? O que seria? A gente está entrando em contato com vocês para fazer um acompanhamento em relação à marca TOTOS que vocês utilizam na empresa. Tá, é para acompanhar? Acompanhamento em relação ao sistema que vocês utilizam, só para verificar se está tudo ok com a utilização e se vocês estão sendo bem a